In [ ]:
# General setup, as explained earlier
import os
from pprint import pprint
from urllib3.util.retry import Retry

import requests
from requests.adapters import HTTPAdapter

PLANET_API_URL = 'https://api.planet.com/data/v1'

def setup_session(api_key=None):
    """
    Initialize a requests.Session that handles Planet api key auth and retries.
    
    :param str api_key:
        A Planet api key. Will be read from the PL_API_KEY env var if not specified.
    
    :returns requests.Session session:
        A Session instance optimized for use with Planet's api.
    """
    if api_key is None:
        api_key = os.getenv('PL_API_KEY')

    session = requests.Session()
    session.auth = (api_key, '')

    retries = Retry(total=5,
                    backoff_factor=0.2,  
                    status_forcelist=[429])

    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session

session = setup_session() # Or pass in an api key if the environment variable isn't set

Searching
---------------

So far, we've been exploring writing filters and getting scene counts using those filters.  Let's use what we've learned about constructing filters to search for scenes that meet those criteria (i.e. get scene IDs instead of just counts).

---

There are two types of searches: 
* **"Quick Search"** `/quick-search`
* **"Saved Searches"** `/searches`

Saved searches are retained on the Planet Platform and may be performed again at any time in the future. You can use these to setup efficient workflows for repetitive tasks, for example, querying an area that is of interest to you, or getting data for specific sensors.

Quick searches are meant to be more fleeting, and are not guaranteed to be available on the API after they are executed.

Searches use the same request format as the `/stats` endpoint except without the `interval` field.

## Quick Search

Let's dive right in and create our first `quick search`:

In [ ]:
# Setup the quick search endpoint url
quick_url = "{}/quick-search".format(PLANET_API_URL)

# Setup Item Types
item_types = ["PSScene4Band"]

# Setup GeoJSON for only imagery that intersects with 40N, 90W
geom = {
    "type": "Point",
    "coordinates": [
        -90,
         40
    ]
}

# Setup a geometry filter
geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": geom
}

# Setup the request
request = {
    "item_types" : item_types,
    "filter" : geometry_filter
}

# Send the POST request to the API quick search endpoint
res = session.post(quick_url, json=request)
res.raise_for_status()
geojson = res.json()

# Print the response
pprint(geojson)

Nice! The response gives us search results for Planet Scope (4 Band) for a specific area.  You might notice that the response looks a lot like GeoJSON -- this is deliberate.  You can treat the `features` key in the response as a sequence of GeoJSON features. 

Let's do another search, this time for Dove R imagery (newer sats) in August in the AOI above:

In [ ]:
# Here we'll find PSScene3Band imagery from Dove R / superdove sats that intersect the point
# 40N, 90W that were acquired in August 2019
item_types = ["PSScene3Band"]

# Setup instrument filter to negate
instrument_filter = {
    "type": "StringInFilter",
    "field_name": "instrument",
    "config": ["PS2.SD"]
}

date_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gt": "2019-08-01T00:00:00Z",
    "lte": "2019-09-01T00:00:00Z"
  }
}

and_filter = {
    "type": "AndFilter",
    "config": [geometry_filter, instrument_filter, date_filter]
}

# Setup the request
request = {
    "item_types" : item_types,
    "filter" : and_filter
}

# Send the POST request to the API quick search endpoint
res = session.post(quick_url, json=request)
res.raise_for_status()
geojson = res.json()

# Print the response
pprint(geojson)

In [ ]:
print('There are {} scenes that match our search'.format(len(geojson['features'])))
for f in geojson['features']:
    # Print the ID for each feature
    print(f["id"])

In [ ]:
# Let's have a look at the first feature, for a reminder of the formatting/etc:
pprint(geojson['features'][0])

## Assets & Permissions

Let's briefly discuss the `_permissions` section.  You may notice that yours looks different than the one on the instructor's display.  Searches will return items even when you don't have permission to download that item.  

As we discussed a couple of sections ago, `assets` are different imagery files, image masks, metadata files or other file types that are associated with the item.  The permissions are generally set at an `asset` level -- you may have permissions to download one asset but not another.

The `_permissions` element in each feature contains a list of assets that the user has access to.  Because this scene is from an AOI that the accounts for this workshop don't have access to, you probably won't have access to many of the assets.  Let's check by looking at:

In [ ]:
pprint(geojson['features'][0]['_permissions'])

Pagination
----------

What happens when there are A LOT of results? 

When the number of matching items exceeds 250, the results are delivered in pages. Let's perform a search query that should return a large number of results:

In [ ]:
# We'll use a point again for brevity -- Polygons are more common in practice!
geom = {
    "type": "Point",
    "coordinates": [
        -90,
         40
    ]
}

# Setup the geometry filter
geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": geom
}

# Setup the request
request = {
    "item_types" : item_types,
    "filter" : geometry_filter
}

# Send the POST request to the API quick search endpoint
res = session.post(quick_url, json=request)
res.raise_for_status()
geojson = res.json()

This will return a lot of results (no date range -- everything in that AOI).  Note that we'll only get 250 results in the first page by default:

In [ ]:
print(len(geojson['features']))

Let's dig into how the results are paginated (i.e. split up into batches).  The next set of results is stored in the `_next` key of the `_links` section:

In [ ]:
pprint(geojson['_links'])

We can request the next batch of 250 results with a `GET` request to the link in `_next`:

In [ ]:
next_geojson = session.get(geojson['_links']['_next']).json()
pprint(next_geojson['features'])

The set of results is finished (or not paginated) when there's not a `_next` key in `_links`.  This means we can iterate through the full set similar to:

In [ ]:
def all_features(item_types, filter_defn):
    """
    Iterate through all results for the specified items (e.g. PSScene3Band) and
    search filter.  Yields a geojson-feature-like dict for each scene returned
    by the search.
    
    :param sequence item_types:
        A list or other sequence of Planet item types.
    :param dict filter_defn:
        A Planet search definition
    """
    request = {'item_types': item_types, 'filter': filter_defn}
    url = "{}/quick-search".format(PLANET_API_URL)
    response = session.post(url, json=request)
    response.raise_for_status()
    body = response.json()
    
    for item in body['features']:
        yield item
    
    next_url = body['_links'].get('_next')
    while next_url:
        response = session.get(next_url)
        response.raise_for_status()
        body = response.json()
        next_url = body['_links'].get('_next')
        for item in body['features']:
            yield item

            
print('Number of PSScene3Band scenes at the point 40N, 90W:')
print(len(list(all_features(item_types, geometry_filter))))

Saved Searches
------------------------

---

The `/searches` endpoint lets you created saved searches that can be reused and that you can optionally enable e-mail alerts for.

To view your saved searches, visit the [`searches/?search_type=saved`](https://api.planet.com/data/v1/searches/?search_type=saved) endpoint.

Finally, let's create a saved search:

In [ ]:
# Setup the saved searches endpoint url
searches_url = "{}/searches".format(PLANET_API_URL)

# Polygon of Vancouver Island
geom = {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -125.29632568359376,
          48.37084770238366
        ],
        [
          -125.29632568359376,
          49.335861591104106
        ],
        [
          -123.2391357421875,
          49.335861591104106
        ],
        [
          -123.2391357421875,
          48.37084770238366
        ],
        [
          -125.29632568359376,
          48.37084770238366
        ]
      ]
    ]
  }

# Setup the geometry filter
geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": geom
}
request = {
    "name" : "Vancouver Island",
    "item_types" : ['PSScene4Band'],
    "filter" : geometry_filter,
}

# Send a POST request to the saved searches endpoint (Create the saved search)
res = session.post(searches_url, json=request)
res.raise_for_status()
our_search = res.json()

# Print the response
pprint(our_search)

We can also list all of our saved searches.  We'll see the one we've created as well as any others that existed before:

In [ ]:
# Send a GET request to the saved searches endpoint with the saved search type parameter (Get saved searches)
res = session.get(searches_url, params={"search_type" : "saved"})
res.raise_for_status()
searches = res.json()["searches"]

for search in searches:
    # Print the ID, Created Time, and Name for each saved search
    print("id: {} created: {} name: {}".format(search["id"], search["created"], search['name']))

Ok, now let's check out the results from a particular saved search.  We'll use the one we just created.  As usual, the `_links` item has the url we'll need to request.  However, you can also construct the url from scatch if you know the search ID:

In [ ]:
# The results url is the search ID
results_url = our_search['_links']['results']
print(results_url)

# But if you know the search's ID, the rest is easy:
print(our_search['id'])
print('{}/searches/{}/results'.format(PLANET_API_URL, our_search['id']))

In [ ]:
# Now let's get the results:
# Send a GET request to the saved search results url (Get the saved search results)
results = session.get(results_url).json()

# Print the number of features in the saved search
print(len(results["features"]))

# Print the first feature in the saved search
pprint(results["features"][0])

A saved search can also be updated by senidng a **`PUT`** request with a changed search definition back to the API.

Did you know that saved searches can also send you a daily email to alert you to when new items are added to the search resutlts? Oh yeah!

In [ ]:
# Remember that weird `__daily_email_enabled` key when we created the search?
pprint(our_search)

In [ ]:
# Let's update it!

# Change the saved search name to "South Vancouver Island"
our_search["name"] = "South Vancouver Island"

# Set the daily email enabled to true (Get email alerts when new items show up in this search)
our_search["__daily_email_enabled"] = True

# Send a PUT request to the saved search's url (the _self link)
our_search_url = our_search['_links']['_self']
res = session.put(our_search_url, json=our_search)
res.raise_for_status()

# ...and just to check that it really worked...  (Also demonstrates the _self link)
pprint(session.get(our_search_url).json())

And finally, let's delete the search we created, so we don't spam ourselves with daily e-mails for Vancouver Island!

In [ ]:
# Make a DELETE request to the search's url to delete it:
res = session.delete(our_search_url)
res.raise_for_status()

# Now let's list our saved searches again and verify that it's not there:
res = session.get(searches_url, params={"search_type" : "saved"})
res.raise_for_status()
searches = res.json()["searches"]

for search in searches:
    # Print the ID, Created Time, and Name for each saved search
    print("id: {} created: {} name: {}".format(search["id"], search["created"], search['name']))